## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lompoc,US,2020-08-01 18:12:39,34.64,-120.46,62.60,82,0,8.05,clear sky
1,1,Busselton,AU,2020-08-01 18:12:11,-33.65,115.33,48.99,82,80,1.77,broken clouds
2,2,Dhidhdhoo,MV,2020-08-01 18:13:03,6.88,73.10,82.85,79,100,14.70,overcast clouds
3,3,Hit,IQ,2020-08-01 18:13:03,33.64,42.83,94.23,13,0,5.57,clear sky
4,4,Nikolskoye,RU,2020-08-01 18:12:14,59.70,30.79,64.40,48,20,6.71,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Dhidhdhoo,MV,2020-08-01 18:13:03,6.88,73.10,82.85,79,100,14.70,overcast clouds
5,5,Mogadishu,SO,2020-08-01 18:12:22,2.04,45.34,77.25,80,40,22.17,light rain
10,10,Albany,US,2020-08-01 18:10:30,42.60,-73.97,90.00,36,41,2.24,scattered clouds
15,15,Hithadhoo,MV,2020-08-01 18:12:25,-0.60,73.08,82.98,74,48,9.69,scattered clouds
18,18,Kuching,MY,2020-08-01 18:13:04,1.55,110.33,78.80,94,20,3.36,few clouds
19,19,Jamestown,US,2020-08-01 18:12:11,42.10,-79.24,78.01,73,20,4.70,few clouds
24,24,Meulaboh,ID,2020-08-01 18:13:05,4.14,96.13,76.05,88,100,4.54,overcast clouds
32,32,Ribeira Grande,PT,2020-08-01 18:12:36,38.52,-28.70,75.20,64,40,6.93,scattered clouds
39,39,Naze,JP,2020-08-01 18:12:38,28.37,129.48,78.80,94,17,2.24,mist
41,41,Saint-Georges,GD,2020-08-01 18:13:07,12.06,-61.75,86.00,79,20,14.99,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                296
City                   296
Country                294
Date                   296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                294
City                   294
Country                294
Date                   294
Lat                    294
Lng                    294
Max Temp               294
Humidity               294
Cloudiness             294
Wind Speed             294
Current Description    294
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Dhidhdhoo,MV,82.85,overcast clouds,6.88,73.10,NaN
5,Mogadishu,SO,77.25,light rain,2.04,45.34,NaN
10,Albany,US,90.00,scattered clouds,42.60,-73.97,NaN
15,Hithadhoo,MV,82.98,scattered clouds,-0.60,73.08,NaN
18,Kuching,MY,78.80,few clouds,1.55,110.33,NaN
19,Jamestown,US,78.01,few clouds,42.10,-79.24,NaN
24,Meulaboh,ID,76.05,overcast clouds,4.14,96.13,NaN
32,Ribeira Grande,PT,75.20,scattered clouds,38.52,-28.70,NaN
39,Naze,JP,78.80,mist,28.37,129.48,NaN
41,Saint-Georges,GD,86.00,few clouds,12.06,-61.75,NaN


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Dhidhdhoo,MV,82.85,overcast clouds,6.88,73.10,Dhidhoo beach
5,Mogadishu,SO,77.25,light rain,2.04,45.34,Hotel Juba
10,Albany,US,90.00,scattered clouds,42.60,-73.97,NaN
15,Hithadhoo,MV,82.98,scattered clouds,-0.60,73.08,Scoop Guest House
18,Kuching,MY,78.80,few clouds,1.55,110.33,Hilton Kuching


In [10]:
hotel_df.count()

City                   294
Country                294
Max Temp               294
Current Description    294
Lat                    294
Lng                    294
Hotel Name             280
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   280
Country                280
Max Temp               280
Current Description    280
Lat                    280
Lng                    280
Hotel Name             280
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))